# Version 3.0: 웨이팅 핫플레이스 RAG 강화 버전

이 노트북은 **RAG(Retrieval-Augmented Generation)**를 활용하여 과거 성공 사례를 학습하는 버전입니다.

## 📌 핵심 특징

### 처리 모드
- 🔹 **단일 매장 모드**: 한 매장씩 상세하게 처리
- 🔹 **다중 매장 모드**: 여러 매장을 일괄 처리 (배치)

### RAG 아키텍처
- 🗄️ **벡터 DB**: Chroma (로컬 파일 기반)
- 📊 **임베딩 모델**: Vertex AI Text Embedding (text-embedding-004, 768차원)
- 🔍 **검색 방식**: 코사인 유사도 기반 Top-K 검색
- 📁 **Collection**: `waiting_hotplace_examples` (독립 컬렉션)
- 💾 **저장 위치**: `./chroma_db` (로컬 디렉토리)

### RAG 워크플로우
1. 새 매장 정보 입력
2. 벡터 DB에서 유사한 과거 사례 검색 (Top-2)
3. 검색된 예시를 프롬프트에 추가
4. LLM이 참고하여 요약문 생성
5. 성공 시 벡터 DB에 저장 (지식 베이스 확장)

### 기대 효과
- ✅ 검증 성공률 향상 (70% → 85%)
- ✅ 일관된 품질 유지
- ✅ 시간이 갈수록 성능 개선
- ✅ 별도 서버 불필요 (로컬 파일)

## 🚀 사용 워크플로우

### 단일 매장 모드
```
1. 섹션 1-5: 환경 설정 (최초 1회)
2. 섹션 6: MODE = "single" 설정 + 매장명 입력
3. 섹션 7: 웹 검색 정보 수집 및 붙여넣기
4. 섹션 8: RAG 강화 프롬프트 실행
5. 섹션 9: 결과 검증
6. 섹션 10: 벡터 DB 저장
```

### 다중 매장 모드
```
1. 섹션 1-5: 환경 설정 (최초 1회)
2. 섹션 6: MODE = "multi" 설정 + SHOP_LIST에 매장 정보 입력
3. 섹션 7: 건너뛰기
4. 섹션 8: RAG 강화 프롬프트 실행 (일괄 처리)
5. 섹션 9: 결과 검증
6. 섹션 10: 벡터 DB 저장
```

## ⚠️ 섹션 1-4: 최초 1회만 실행 (수정 금지)

In [ ]:
# 섹션 1: Package 설치
%pip install google-cloud-aiplatform google-genai python-dotenv chromadb

In [ ]:
# 섹션 2: 라이브러리 Import
import os
import json
import pandas as pd
import vertexai
from google import genai
from google.genai import types
from dotenv import load_dotenv
import chromadb
from datetime import datetime
from vertexai.language_models import TextEmbeddingModel

In [ ]:
# 섹션 3: Vertex AI 초기화
load_dotenv()

PROJECT_ID = "wad-dw"
LOCATION = "us-central1"

vertexai.init(project=PROJECT_ID, location=LOCATION)
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

print(f"✅ Vertex AI 초기화 완료: {PROJECT_ID} ({LOCATION})")

In [ ]:
# 섹션 4: Chroma 벡터 DB 초기화 (공통 벡터 DB 사용)
try:
    # 공통 벡터 DB 경로: waiting_hotplace/../chroma_db = shop_summary/chroma_db
    chroma_client = chromadb.PersistentClient(path="../chroma_db")
    
    # 카테고리별 독립 컬렉션 (코사인 유사도 사용)
    waiting_hotplace_collection = chroma_client.get_or_create_collection(
        name="waiting_hotplace_examples",
        metadata={"hnsw:space": "cosine"}  # 코사인 유사도
    )
    
    print(f"✅ Chroma 벡터 DB 초기화 완료 (공통 벡터 DB)")
    print(f"   - 저장 위치: ../chroma_db (shop_summary/chroma_db)")
    print(f"   - Collection: waiting_hotplace_examples")
    print(f"   - 저장된 예시: {waiting_hotplace_collection.count()}개")
    print(f"\n   💡 knowledge_base/에서 최초 1회 인덱싱 후, 모든 카테고리가 공유합니다")
    
except Exception as e:
    print(f"❌ Chroma 초기화 실패: {e}")
    chroma_client = None
    waiting_hotplace_collection = None

## ✅ 섹션 5: RAG 함수 정의

In [ ]:
# ========================================
# RAG 핵심 함수
# ========================================

def generate_embedding(text, model="text-embedding-004"):
    """텍스트를 768차원 벡터로 변환"""
    try:
        embedding_model = TextEmbeddingModel.from_pretrained(model)
        embeddings = embedding_model.get_embeddings([text])
        
        return embeddings[0].values
    except Exception as e:
        print(f"❌ 임베딩 생성 실패: {e}")
        return None


def retrieve_similar_examples(query_text, collection, top_k=2):
    """벡터 DB에서 유사한 예시 검색"""
    if not collection:
        print("⚠️ Chroma 연결 없음 - RAG 건너뛰기")
        return []
    
    # 컬렉션이 비어있는지 확인
    if collection.count() == 0:
        return []
    
    try:
        # 쿼리 임베딩 생성
        query_embedding = generate_embedding(query_text)
        if not query_embedding:
            return []
        
        # Chroma 벡터 검색
        results = collection.query(
            query_embeddings=[query_embedding],
            n_results=min(top_k, collection.count()),  # 저장된 개수보다 많이 요청하지 않음
            include=["documents", "metadatas", "distances"]
        )
        
        # Chroma 결과 형식 변환
        formatted_results = []
        
        # results는 배치 형식: {'ids': [[...]], 'documents': [[...]], ...}
        if results['ids'] and len(results['ids'][0]) > 0:
            for i in range(len(results['ids'][0])):
                metadata = results['metadatas'][0][i]
                distance = results['distances'][0][i]
                
                # 코사인 거리를 유사도로 변환 (distance 범위: 0~2)
                # distance 0 = 동일, distance 2 = 정반대
                # similarity = 1 - (distance / 2)
                similarity = 1.0 - (distance / 2.0)
                
                formatted_results.append({
                    "shop_name": metadata.get("shop_name"),
                    "title": metadata.get("title"),
                    "summaries": json.loads(metadata.get("summaries", "[]")),
                    "collected_info": results['documents'][0][i],
                    "score": similarity
                })
        
        return formatted_results
        
    except Exception as e:
        print(f"⚠️ 검색 실패: {e}")
        return []


def format_rag_context(similar_examples):
    """검색된 예시를 프롬프트용 컨텍스트로 포맷"""
    if not similar_examples:
        return "[참고 가능한 과거 사례 없음]"
    
    context = "[과거 성공 사례 참고]\n\n"
    
    for idx, example in enumerate(similar_examples, 1):
        context += f"**참고 사례 {idx}: {example.get('shop_name')}**\n"
        context += f"제목: {example.get('title')}\n"
        context += f"요약문:\n"
        for i, summary in enumerate(example.get('summaries', []), 1):
            context += f"  {i}. {summary}\n"
        context += f"\n유사도 점수: {example.get('score', 0):.3f}\n"
        context += "─" * 80 + "\n\n"
    
    return context


def store_successful_example(collection, shop_seq, shop_name, collected_info, 
                            title, summaries, category="waiting_hotplace"):
    """성공한 예시를 벡터 DB에 저장"""
    if not collection:
        print("⚠️ Chroma 연결 없음 - 저장 건너뛰기")
        return False
    
    try:
        # 임베딩 생성
        info_embedding = generate_embedding(collected_info)
        
        if not info_embedding:
            print("❌ 임베딩 생성 실패")
            return False
        
        # 고유 ID 생성
        doc_id = f"{category}_{shop_seq}_{datetime.utcnow().strftime('%Y%m%d%H%M%S')}"
        
        # Chroma에 저장
        collection.add(
            documents=[collected_info],  # 원본 텍스트
            embeddings=[info_embedding],  # 768차원 벡터
            metadatas=[{
                "shop_seq": shop_seq,
                "shop_name": shop_name,
                "category": category,
                "title": title,
                "summaries": json.dumps(summaries, ensure_ascii=False),  # JSON 문자열로 저장
                "created_at": datetime.utcnow().isoformat()
            }],
            ids=[doc_id]
        )
        
        print(f"✅ 벡터 DB 저장 완료: {shop_name}")
        return True
        
    except Exception as e:
        print(f"❌ 저장 실패: {e}")
        return False


print("✅ RAG 함수 정의 완료")
print("   - generate_embedding(): 텍스트 → 768차원 벡터")
print("   - retrieve_similar_examples(): 유사 예시 검색 (Chroma)")
print("   - format_rag_context(): 프롬프트 포맷팅")
print("   - store_successful_example(): 벡터 DB 저장 (Chroma)")

## ✅ 섹션 6: 매장 정보 입력 (수정 가능)

### 처리 모드 선택

#### 🔹 단일 매장 모드 (single)
- 한 매장씩 상세하게 처리
- 섹션 7에서 정보 수집
- 결과: JSON 파일 1개

#### 🔹 다중 매장 모드 (multi)
- 여러 매장을 일괄 처리
- SHOP_LIST에 모든 매장 정보 입력
- 섹션 7 건너뛰기
- 결과: JSON + CSV 파일

In [ ]:
# ========================================
# 매장 정보 입력 (다중 매장 모드)
# ========================================

# 1️⃣ 매장 기본 정보 (shop_seq, shop_name)
SHOPS = [
    (1, "성수연방"),
    (2, "르바게트"),
    (3, "담소사골순대"),
]

# 2️⃣ 매장별 수집된 정보 (매장명을 키로 사용)
COLLECTED_INFO = {
    "성수연방": "",
    "르바게트": "",
    "담소사골순대": "",
}

# 3️⃣ SHOP_LIST 자동 생성 (shop_seq, shop_name, collected_info 매핑)
SHOP_LIST = [
    {
        "shop_seq": seq,
        "shop_name": name,
        "collected_info": COLLECTED_INFO.get(name, "[정보 없음 - COLLECTED_INFO에 추가 필요]")
    }
    for seq, name in SHOPS
]

# 출력
print(f"📍 다중 매장 모드")
print(f"   - 총 매장 수: {len(SHOP_LIST)}개")
for idx, shop in enumerate(SHOP_LIST, 1):
    info_length = len(shop['collected_info'].strip())
    status = "✅" if info_length > 100 else "⚠️"
    print(f"   {status} {idx}. {shop['shop_name']} (seq: {shop['shop_seq']}) - {info_length:,}자")

## ✅ 섹션 7: 웹 검색 및 정보 수집

### 정보 수집 방법

이 노트북은 **외부 공식 정보**를 기반으로 요약문을 생성합니다:

- 미쉐린 가이드 (guide.michelin.com.kr)
- 블루리번 (bluer.co.kr)
- 셰프 인터뷰
- 코스 구성 설명
- 철학 / 공간 / 전통

### 수집 프로세스

1. 섹션 6에서 생성된 검색 쿼리 사용
2. 웹에서 정보 검색 및 복사
3. 아래 `collected_info` 변수에 붙여넣기
4. 섹션 8 실행하여 요약문 생성

In [ ]:
# 이 섹션은 건너뜁니다.
# 섹션 6의 COLLECTED_INFO 딕셔너리에서 이미 정보를 입력했습니다.

print("✅ 다중 매장 모드")
print("   섹션 6의 COLLECTED_INFO에 매장별 정보를 입력하세요.")
print("   입력 완료 후 바로 섹션 8을 실행하세요.")

## ✅ 섹션 8: RAG 강화 프롬프트 실행

In [ ]:
# ========================================
# RAG 강화 프롬프트 실행
# ========================================

# 기본 시스템 프롬프트 (모든 매장에 공통)
system_prompt = """
당신은 미쉐린 가이드와 블루리번 수준의 파인다이닝 레스토랑 전문 작가입니다.
외부 공식 정보를 바탕으로 매장의 정체성을 압축적으로 표현하는 요약문을 작성합니다.

## 작성 원칙
1. 사실 기반: 공식 출처의 정보만 사용
2. 구조화: 제목 1개 + 요약 문장 3개
3. 간결성: 각 문장 약 40-60자
4. 전문성: 미식 용어 정확히 사용
5. 객관성: 과장 금지
6. 독창성: 예시를 그대로 복사하지 말고, 톤앤매너만 참고하여 해당 매장만의 고유한 특징을 작성

## 요약 문장 구조
- **1번 문장**: 셰프 철학 / 브랜드 정체성 / 조리 철학
- **2번 문장**: 코스 구성 / 시그니처 메뉴 / 조리 기법
- **3번 문장**: 공간 특징 / 분위기 / 미식 경험

## 제목 작성 규칙
- 형식: "[공간 특징] 속, [시그니처 요소]를 [동사]하는 [장르]"
- 길이: 15-30자

## 출력 형식
{{
  "shop_seq": 숫자,
  "shop_name": "매장명",
  "title": "제목",
  "summaries": ["문장1", "문장2", "문장3"]
}}
"""

# 모델 설정
MODEL_NAME = "gemini-2.5-pro"
generation_config = types.GenerateContentConfig(
    temperature=0.6,  # 웨이팅 핫플레이스는 더 창의적
    max_output_tokens=4096,
    response_mime_type="application/json"
)

print(f"🤖 모델: {MODEL_NAME}")
print(f"   - Temperature: 0.6 (높은 창의성)")
print(f"   - Max Output Tokens: 4,096\n")

# -----------------------------------------
# 요약문 생성 함수 (RAG 강화)
# -----------------------------------------
def generate_summary_with_rag(shop_seq, shop_name, collected_info):
    """RAG를 활용한 단일 매장 요약문 생성"""
    
    # 1. 유사한 과거 사례 검색
    similar_examples = retrieve_similar_examples(
        query_text=collected_info,
        collection=waiting_hotplace_collection,
        top_k=2
    )
    
    # 2. RAG 컨텍스트 생성
    rag_context = format_rag_context(similar_examples)
    
    # 3. 유저 프롬프트 생성 (RAG 컨텍스트 포함)
    user_prompt = f"""
    {rag_context}

    [매장 정보]
    매장명: {shop_name}
    Shop Seq: {shop_seq}

    [수집된 정보]
    {collected_info}

    위 과거 사례들의 스타일을 참고하되, {shop_name} 매장만의 고유한 특징을 작성하세요.
    """
    
    try:
        # 4. LLM 호출
        response = client.models.generate_content(
            model=MODEL_NAME,
            contents=f"{system_prompt}\n\n{user_prompt}",
            config=generation_config
        )
        
        # 5. JSON 파싱
        result_text = response.text
        result_json = json.loads(result_text)
        
        # 6. 토큰 사용량 추출
        usage_metadata = response.usage_metadata if hasattr(response, 'usage_metadata') else None
        token_info = {}
        if usage_metadata:
            token_info = {
                "input_tokens": usage_metadata.prompt_token_count,
                "output_tokens": usage_metadata.candidates_token_count,
                "total_tokens": usage_metadata.total_token_count
            }
        
        return {
            "status": "success",
            "data": result_json,
            "tokens": token_info,
            "similar_count": len(similar_examples)
        }
        
    except json.JSONDecodeError as e:
        return {
            "status": "error",
            "error_type": "JSONDecodeError",
            "message": str(e),
            "response": response.text if 'response' in locals() else None,
            "tokens": {},
            "similar_count": len(similar_examples)
        }
        
    except Exception as e:
        return {
            "status": "error",
            "error_type": type(e).__name__,
            "message": str(e),
            "response": response.text if 'response' in locals() else None,
            "tokens": {},
            "similar_count": len(similar_examples)
        }

# -----------------------------------------
# 단일/다중 모드 처리
# -----------------------------------------
all_results = []
total_input_tokens = 0
total_output_tokens = 0
total_tokens = 0

if MODE == "single":
    # 단일 매장 모드
    print(f"🔍 [{SHOP_NAME}] 유사 사례 검색 중...\n")
    
    result = generate_summary_with_rag(SHOP_SEQ, SHOP_NAME, collected_info)
    
    print(f"   ✅ {result['similar_count']}개 유사 사례 발견\n")
    print(f"📝 요약문 생성 중...\n")
    
    if result["status"] == "success":
        result_json = result["data"]
        all_results.append(result_json)
        
        # 토큰 사용량 출력
        tokens = result.get("tokens", {})
        if tokens:
            print(f"📊 토큰 사용량:")
            print(f"   - Input:  {tokens.get('input_tokens', 0):,} tokens")
            print(f"   - Output: {tokens.get('output_tokens', 0):,} tokens")
            print(f"   - Total:  {tokens.get('total_tokens', 0):,} tokens\n")
        
        print("✅ 요약문 생성 완료\n")
        print("=" * 80)
        print(json.dumps(result_json, ensure_ascii=False, indent=2))
        print("=" * 80)
    else:
        print(f"❌ 에러 발생: {result['error_type']}")
        print(f"   메시지: {result['message']}")
        if result.get('response'):
            print(f"   응답: {result['response'][:200]}...")

elif MODE == "multi":
    # 다중 매장 모드
    print(f"📝 {len(SHOP_LIST)}개 매장 RAG 강화 요약문 생성 시작\n")
    print("=" * 80)
    
    for idx, shop in enumerate(SHOP_LIST, 1):
        shop_seq = shop["shop_seq"]
        shop_name = shop["shop_name"]
        collected_info = shop["collected_info"]
        
        print(f"\n[{idx}/{len(SHOP_LIST)}] {shop_name} 처리 중...")
        print(f"   🔍 유사 사례 검색 중...")
        
        result = generate_summary_with_rag(shop_seq, shop_name, collected_info)
        
        if result["status"] == "success":
            result_json = result["data"]
            all_results.append(result_json)
            
            # 토큰 사용량 출력 및 누적
            tokens = result.get("tokens", {})
            if tokens:
                input_tokens = tokens.get('input_tokens', 0)
                output_tokens = tokens.get('output_tokens', 0)
                total_tokens_shop = tokens.get('total_tokens', 0)
                
                total_input_tokens += input_tokens
                total_output_tokens += output_tokens
                total_tokens += total_tokens_shop
                
                print(f"   ✅ 성공 | RAG: {result['similar_count']}개 | Input: {input_tokens:,} | Output: {output_tokens:,} | Total: {total_tokens_shop:,} tokens")
            else:
                print(f"   ✅ 성공 | RAG: {result['similar_count']}개")
        else:
            print(f"   ❌ 실패: {result['error_type']} - {result['message']}")
            # 실패한 경우에도 기록
            all_results.append({
                "shop_seq": shop_seq,
                "shop_name": shop_name,
                "status": "error",
                "error": result['message']
            })
    
    print("\n" + "=" * 80)
    print(f"\n✅ 전체 완료: {len(all_results)}개 매장 처리")
    
    # 성공/실패 개수
    success_count = sum(1 for r in all_results if r.get("status") != "error")
    error_count = len(all_results) - success_count
    print(f"   - 성공: {success_count}개")
    if error_count > 0:
        print(f"   - 실패: {error_count}개")
    
    # 총 토큰 사용량 출력
    if total_tokens > 0:
        print(f"\n📊 총 토큰 사용량:")
        print(f"   - Total Input:  {total_input_tokens:,} tokens")
        print(f"   - Total Output: {total_output_tokens:,} tokens")
        print(f"   - Total:        {total_tokens:,} tokens")
    
    # -----------------------------------------
    # 전체 결과 상세 출력
    # -----------------------------------------
    print("\n" + "=" * 80)
    print("📋 전체 결과 상세보기")
    print("=" * 80)
    
    for idx, result in enumerate(all_results, 1):
        print(f"\n{'─' * 80}")
        print(f"[{idx}] {result.get('shop_name')}")
        print(f"{'─' * 80}")
        
        if result.get("status") != "error":
            # 성공한 경우 - 전체 내용 출력
            print(f"\n📌 제목:")
            print(f"   {result.get('title')}")
            
            print(f"\n📝 요약문:")
            summaries = result.get('summaries', [])
            for i, summary in enumerate(summaries, 1):
                print(f"   {i}. {summary}")
            
            print(f"\n✅ 상태: 성공")
        else:
            # 실패한 경우
            print(f"\n❌ 상태: 에러")
            print(f"   오류 내용: {result.get('error', 'N/A')}")
    
    print("\n" + "=" * 80)

## ✅ 섹션 9: 결과 검증

In [ ]:
# ========================================
# 간단한 검증
# ========================================

if len(all_results) == 0:
    print("⚠️ 검증할 결과가 없습니다.")
    print("   먼저 섹션 8을 실행하여 요약문을 생성하세요.")
    validation_results = []
else:
    print("🔍 요약문 검증 시작\n")
    print("=" * 80)
    
    validation_results = []
    
    for idx, result in enumerate(all_results, 1):
        # 에러 상태인 결과는 스킵
        if result.get("status") == "error":
            print(f"\n[{idx}] {result.get('shop_name')}: 생성 실패로 검증 스킵")
            validation_results.append({
                "shop_name": result.get('shop_name'),
                "passed": False,
                "error": "생성 실패"
            })
            continue
        
        shop_name = result.get("shop_name", "Unknown")
        print(f"\n[{idx}] {shop_name}")
        print("─" * 80)
        
        # 구조 검증
        is_valid = (
            "title" in result and
            "summaries" in result and
            len(result.get('summaries', [])) == 3
        )
        
        if is_valid:
            print("✅ 검증 통과: 구조가 올바릅니다")
            
            # 문장 길이 확인
            summaries = result.get('summaries', [])
            for i, summary in enumerate(summaries, 1):
                length = len(summary)
                if length < 30:
                    print(f"   ⚠️ {i}번 문장이 너무 짧음: {length}자 (권장 40-60자)")
                elif length > 100:
                    print(f"   ⚠️ {i}번 문장이 너무 김: {length}자 (권장 40-60자)")
            
            validation_results.append({
                "shop_name": shop_name,
                "passed": True
            })
        else:
            print("❌ 검증 실패: 구조 오류")
            validation_results.append({
                "shop_name": shop_name,
                "passed": False,
                "error": "구조 오류"
            })
    
    # 전체 검증 요약
    print("\n" + "=" * 80)
    print("📊 검증 요약")
    print("=" * 80)
    
    passed_count = sum(1 for v in validation_results if v.get('passed', False))
    failed_count = len(validation_results) - passed_count
    
    print(f"✅ 통과: {passed_count}개")
    if failed_count > 0:
        print(f"❌ 실패: {failed_count}개")
    
    if len(validation_results) > 0:
        success_rate = (passed_count / len(validation_results)) * 100
        print(f"\n성공률: {success_rate:.1f}%")

## ✅ 섹션 10: 벡터 DB 저장

In [ ]:
# ========================================
# 벡터 DB 저장 및 결과 파일 저장
# ========================================

if len(validation_results) == 0:
    print("⚠️ 저장할 결과가 없습니다.")
    print("   먼저 섹션 8을 실행하여 요약문을 생성하세요.")
else:
    print("\n💾 벡터 DB에 저장 중...\n")
    print("=" * 80)
    
    stored_count = 0
    
    # 검증 통과한 결과만 벡터 DB에 저장
    for idx, (result, validation) in enumerate(zip(all_results, validation_results), 1):
        if not validation.get('passed', False):
            print(f"[{idx}] {validation.get('shop_name')}: 검증 실패로 저장 스킵")
            continue
        
        shop_seq = result.get('shop_seq')
        shop_name = result.get('shop_name')
        title = result.get('title')
        summaries = result.get('summaries')
        
        # collected_info 가져오기
        if MODE == "single":
            info = collected_info
        elif MODE == "multi":
            # SHOP_LIST에서 해당 매장의 collected_info 찾기
            shop_data = next((s for s in SHOP_LIST if s['shop_seq'] == shop_seq), None)
            info = shop_data['collected_info'] if shop_data else ""
        
        # 벡터 DB에 저장
        success = store_successful_example(
            collection=waiting_hotplace_collection,
            shop_seq=shop_seq,
            shop_name=shop_name,
            collected_info=info,
            title=title,
            summaries=summaries,
            category="waiting_hotplace"
        )
        
        if success:
            stored_count += 1
            print(f"   ✅ [{idx}] {shop_name} 저장 완료")
        else:
            print(f"   ❌ [{idx}] {shop_name} 저장 실패")
    
    print("\n" + "=" * 80)
    print(f"✅ 벡터 DB 저장 완료: {stored_count}개 매장")
    print(f"   - 현재 저장된 예시: {waiting_hotplace_collection.count()}개")
    print(f"   - 다음 매장부터 이 사례들을 참고하여 품질이 향상됩니다!")
    
    # -----------------------------------------
    # 파일 저장 (선택 사항)
    # -----------------------------------------
    from datetime import datetime
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    if MODE == "single":
        # 단일 매장: JSON 파일로 저장
        result_json = all_results[0] if all_results else {}
        filename = f"summary_rag_{result_json.get('shop_name', 'unknown')}_{timestamp}.json"
        
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(result_json, f, ensure_ascii=False, indent=2)
        
        print(f"\n📁 JSON 저장 완료: {filename}")
        print(f"   - 매장명: {result_json.get('shop_name')}")
        print(f"   - 제목: {result_json.get('title')}")
    
    elif MODE == "multi":
        # 다중 매장: JSON + CSV 파일로 저장
        
        # 1. JSON 파일
        json_filename = f"summaries_rag_batch_{timestamp}.json"
        with open(json_filename, 'w', encoding='utf-8') as f:
            json.dump(all_results, f, ensure_ascii=False, indent=2)
        
        print(f"\n📁 JSON 저장 완료: {json_filename}")
        print(f"   - 총 매장 수: {len(all_results)}개")
        
        # 2. CSV 파일
        csv_data = []
        for result in all_results:
            if result.get("status") != "error":
                csv_data.append({
                    "shop_seq": result.get("shop_seq"),
                    "shop_name": result.get("shop_name"),
                    "title": result.get("title"),
                    "summary_1": result.get("summaries", [])[0] if len(result.get("summaries", [])) > 0 else "",
                    "summary_2": result.get("summaries", [])[1] if len(result.get("summaries", [])) > 1 else "",
                    "summary_3": result.get("summaries", [])[2] if len(result.get("summaries", [])) > 2 else "",
                    "status": "success"
                })
            else:
                csv_data.append({
                    "shop_seq": result.get("shop_seq"),
                    "shop_name": result.get("shop_name"),
                    "title": "",
                    "summary_1": "",
                    "summary_2": "",
                    "summary_3": "",
                    "status": "error",
                    "error": result.get("error", "")
                })
        
        df = pd.DataFrame(csv_data)
        csv_filename = f"summaries_rag_batch_{timestamp}.csv"
        df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
        
        print(f"📁 CSV 저장 완료: {csv_filename}")
        
        # 통계
        success_count = len(df[df['status'] == 'success'])
        error_count = len(df[df['status'] == 'error'])
        
        print(f"\n📊 통계:")
        print(f"   - 성공: {success_count}개")
        if error_count > 0:
            print(f"   - 실패: {error_count}개")